# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [1]:
import numpy as np
import pandas as pd

# Object creation

In [2]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [3]:
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [4]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [5]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-23,-0.122144,-1.784505,1.192836,1.229404
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048
2022-06-25,-2.155141,0.576865,2.097393,1.638619
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-28,0.324334,0.365928,-0.101784,-0.318715


In [6]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


In [7]:
# the datatype of each of the columns would be different
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
G            object
dtype: object

# Viewing Data

In [8]:
# top of the dataframe
df.head()

,A,B,C,D
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-23,-0.122144,-1.784505,1.192836,1.229404


In [9]:
# bottom 3 records of the dataframe
df.tail(3)

,A,B,C,D
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-28,0.324334,0.365928,-0.101784,-0.318715


In [10]:
# index of the df
df.index

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# columns of the df
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

array([[-0.49425473, -0.98893615,  0.59988598, -0.23842126],
       [-0.35697266,  0.24114981, -0.21507967,  1.46564809],
       [ 1.81994213, -2.25500051, -0.26940875,  0.97519255],
       [ 2.03959541,  0.3881543 , -0.58322127, -0.58501082],
       [-0.12214376, -1.78450493,  1.19283605,  1.22940449],
       [ 1.08461765, -1.30254733, -0.78750146, -0.22904784],
       [-2.15514111,  0.57686533,  2.09739334,  1.63861928],
       [ 0.8771015 ,  0.21472822,  0.74255281,  1.0988014 ],
       [ 1.52623612,  2.27371813, -0.4302674 ,  0.31077055],
       [ 0.32433427,  0.36592751, -0.1017842 , -0.31871471]])

In [13]:
df2.to_numpy()

array([[1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo']], dtype=object)

In [14]:
# quick summary stats
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.454331,-0.227045,0.224541,0.534724
std,1.283205,1.346601,0.913091,0.836222
min,-2.155141,-2.255001,-0.787501,-0.585011
25%,-0.298265,-1.224145,-0.390053,-0.236078
50%,0.600718,0.227939,-0.158432,0.642982
75%,1.415832,0.382598,0.706886,1.196754
max,2.039595,2.273718,2.097393,1.638619


1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [15]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.454331,-0.227045,0.224541,0.534724
std,1.283205,1.346601,0.913091,0.836222
min,-2.155141,-2.255001,-0.787501,-0.585011
10%,-0.660343,-1.831554,-0.603649,-0.345344
50%,0.600718,0.227939,-0.158432,0.642982
90%,1.841907,0.746551,1.283292,1.482945
max,2.039595,2.273718,2.097393,1.638619


In [16]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

,A,C,D
count,6.0,6.0,6.0
mean,1.0,1.0,3.0
std,0.0,0.0,0.0
min,1.0,1.0,3.0
25%,1.0,1.0,3.0
50%,1.0,1.0,3.0
75%,1.0,1.0,3.0
max,1.0,1.0,3.0


In [17]:
# transpose the data
# turn rows to columns and vice versa
df.T

,2022-06-19,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28
A,-0.494255,-0.356973,1.819942,2.039595,-0.122144,1.084618,-2.155141,0.877102,1.526236,0.324334
B,-0.988936,0.241150,-2.255001,0.388154,-1.784505,-1.302547,0.576865,0.214728,2.273718,0.365928
C,0.599886,-0.215080,-0.269409,-0.583221,1.192836,-0.787501,2.097393,0.742553,-0.430267,-0.101784
D,-0.238421,1.465648,0.975193,-0.585011,1.229404,-0.229048,1.638619,1.098801,0.310771,-0.318715


### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [18]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2022-06-19,-0.238421,0.599886,-0.988936,-0.494255
2022-06-20,1.465648,-0.215080,0.241150,-0.356973
2022-06-21,0.975193,-0.269409,-2.255001,1.819942
2022-06-22,-0.585011,-0.583221,0.388154,2.039595
2022-06-23,1.229404,1.192836,-1.784505,-0.122144
2022-06-24,-0.229048,-0.787501,-1.302547,1.084618
2022-06-25,1.638619,2.097393,0.576865,-2.155141
2022-06-26,1.098801,0.742553,0.214728,0.877102
2022-06-27,0.310771,-0.430267,2.273718,1.526236
2022-06-28,-0.318715,-0.101784,0.365928,0.324334


In [19]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2022-06-28,0.324334,0.365928,-0.101784,-0.318715
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-25,-2.155141,0.576865,2.097393,1.638619
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048
2022-06-23,-0.122144,-1.784505,1.192836,1.229404
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421


In [20]:
# sort ascending by values in a column
df.sort_values(by="A")

,A,B,C,D
2022-06-25,-2.155141,0.576865,2.097393,1.638619
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-23,-0.122144,-1.784505,1.192836,1.229404
2022-06-28,0.324334,0.365928,-0.101784,-0.318715
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011


In [21]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

,A,B,C,D,E,F,G
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo


In [22]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [23]:
# selecting a single column returns a Series object
df["A"]

2022-06-19   -0.494255
2022-06-20   -0.356973
2022-06-21    1.819942
2022-06-22    2.039595
2022-06-23   -0.122144
2022-06-24    1.084618
2022-06-25   -2.155141
2022-06-26    0.877102
2022-06-27    1.526236
2022-06-28    0.324334
Freq: D, Name: A, dtype: float64

In [24]:
# selecting a slice
df[1:5]

,A,B,C,D
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-23,-0.122144,-1.784505,1.192836,1.229404


## Selection by label

In [25]:
# selecting based on a label
df.loc[dates[0]]

A   -0.494255
B   -0.988936
C    0.599886
D   -0.238421
Name: 2022-06-19 00:00:00, dtype: float64

In [26]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

,A,B
2022-06-19,-0.494255,-0.988936
2022-06-20,-0.356973,0.241150
2022-06-21,1.819942,-2.255001
2022-06-22,2.039595,0.388154
2022-06-23,-0.122144,-1.784505
2022-06-24,1.084618,-1.302547
2022-06-25,-2.155141,0.576865
2022-06-26,0.877102,0.214728
2022-06-27,1.526236,2.273718
2022-06-28,0.324334,0.365928


In [27]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

A   -0.356973
B    0.241150
C   -0.215080
D    1.465648
Name: 2022-06-20 00:00:00, dtype: float64
res.shape =  (4,)  vs. df.shape =  (10, 4)


In [28]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

-0.4942547262148707

In [29]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

-0.4942547262148707

## Selection by position

In [30]:
df.iloc[2]

A    1.819942
B   -2.255001
C   -0.269409
D    0.975193
Name: 2022-06-21 00:00:00, dtype: float64

In [31]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

,A,B
2022-06-20,-0.356973,0.241150
2022-06-21,1.819942,-2.255001
2022-06-22,2.039595,0.388154
2022-06-23,-0.122144,-1.784505


In [32]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

,A,C
2022-06-19,-0.494255,0.599886
2022-06-20,-0.356973,-0.215080
2022-06-21,1.819942,-0.269409
2022-06-25,-2.155141,2.097393


In [33]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

,C,B,A,C
2022-06-19,0.599886,-0.988936,-0.494255,0.599886
2022-06-20,-0.215080,0.241150,-0.356973,-0.215080
2022-06-21,-0.269409,-2.255001,1.819942,-0.269409
2022-06-25,2.097393,0.576865,-2.155141,2.097393


In [34]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

,A,B,C,D
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-21,1.819942,-2.255001,-0.269409,0.975193


In [35]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

,C,D
2022-06-19,0.599886,-0.238421
2022-06-20,-0.215080,1.465648
2022-06-21,-0.269409,0.975193
2022-06-22,-0.583221,-0.585011
2022-06-23,1.192836,1.229404
2022-06-24,-0.787501,-0.229048
2022-06-25,2.097393,1.638619
2022-06-26,0.742553,1.098801
2022-06-27,-0.430267,0.310771
2022-06-28,-0.101784,-0.318715


In [36]:
# everything, because you can
df.iloc[:, :]

,A,B,C,D
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421
2022-06-20,-0.356973,0.241150,-0.215080,1.465648
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-23,-0.122144,-1.784505,1.192836,1.229404
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048
2022-06-25,-2.155141,0.576865,2.097393,1.638619
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-28,0.324334,0.365928,-0.101784,-0.318715


In [37]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

-0.21507967086060592

In [38]:
#
# method two: use iat
df.iat[1, 2]

-0.21507967086060592

## Boolean Indexing

In [39]:
# use a value found in a single col to get data
df[df["A"] > 0]

,A,B,C,D
2022-06-21,1.819942,-2.255001,-0.269409,0.975193
2022-06-22,2.039595,0.388154,-0.583221,-0.585011
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-27,1.526236,2.273718,-0.430267,0.310771
2022-06-28,0.324334,0.365928,-0.101784,-0.318715


In [40]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

,A,B,C,D
2022-06-19,NaN,NaN,0.599886,NaN
2022-06-20,NaN,0.241150,NaN,1.465648
2022-06-21,1.819942,NaN,NaN,0.975193
2022-06-22,2.039595,0.388154,NaN,NaN
2022-06-23,NaN,NaN,1.192836,1.229404
2022-06-24,1.084618,NaN,NaN,NaN
2022-06-25,NaN,0.576865,2.097393,1.638619
2022-06-26,0.877102,0.214728,0.742553,1.098801
2022-06-27,1.526236,2.273718,NaN,0.310771
2022-06-28,0.324334,0.365928,NaN,NaN


In [41]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

,A,B,C,D,E
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,one
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,two
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,three
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,four
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,two
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,five
2022-06-25,-2.155141,0.576865,2.097393,1.638619,one
2022-06-26,0.877102,0.214728,0.742553,1.098801,two
2022-06-27,1.526236,2.273718,-0.430267,0.310771,three
2022-06-28,0.324334,0.365928,-0.101784,-0.318715,four


In [42]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

,A,B,C,D,E
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,two
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,two
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,five
2022-06-26,0.877102,0.214728,0.742553,1.098801,two


## Setting values

In [43]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

2022-06-19    11
2022-06-20    12
2022-06-21    13
2022-06-22    14
2022-06-23    15
2022-06-24    16
2022-06-25    17
2022-06-26    18
2022-06-27    19
2022-06-28    20
Freq: D, dtype: int64

In [44]:
df["F"] = s1
df

,A,B,C,D,F
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,12
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,15
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,16
2022-06-25,-2.155141,0.576865,2.097393,1.638619,17
2022-06-26,0.877102,0.214728,0.742553,1.098801,18
2022-06-27,1.526236,2.273718,-0.430267,0.310771,19
2022-06-28,0.324334,0.365928,-0.101784,-0.318715,20


In [45]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

,A,B,C,D,F
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,12
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,15
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,16
2022-06-25,-2.155141,0.576865,2.097393,1.638619,17
2022-06-26,0.877102,0.214728,0.742553,1.098801,18
2022-06-27,1.526236,2.273718,-0.430267,0.310771,19
2022-06-28,0.324334,0.365928,-0.101784,-0.318715,20


In [46]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.599886,-0.238421,11
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,12
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,15
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,16
2022-06-25,-2.155141,0.576865,2.097393,1.638619,17
2022-06-26,0.877102,0.214728,0.742553,1.098801,18
2022-06-27,1.526236,2.273718,-0.430267,0.310771,19
2022-06-28,0.324334,0.365928,-0.101784,-0.318715,20


In [47]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.599886,5,11
2022-06-20,-0.356973,0.241150,-0.215080,5,12
2022-06-21,1.819942,-2.255001,-0.269409,5,13
2022-06-22,2.039595,0.388154,-0.583221,5,14
2022-06-23,-0.122144,-1.784505,1.192836,5,15
2022-06-24,1.084618,-1.302547,-0.787501,5,16
2022-06-25,-2.155141,0.576865,2.097393,5,17
2022-06-26,0.877102,0.214728,0.742553,5,18
2022-06-27,1.526236,2.273718,-0.430267,5,19
2022-06-28,0.324334,0.365928,-0.101784,5,20


In [48]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.599886,-5,-11
2022-06-20,-0.356973,-0.241150,-0.215080,-5,-12
2022-06-21,-1.819942,-2.255001,-0.269409,-5,-13
2022-06-22,-2.039595,-0.388154,-0.583221,-5,-14
2022-06-23,-0.122144,-1.784505,-1.192836,-5,-15
2022-06-24,-1.084618,-1.302547,-0.787501,-5,-16
2022-06-25,-2.155141,-0.576865,-2.097393,-5,-17
2022-06-26,-0.877102,-0.214728,-0.742553,-5,-18
2022-06-27,-1.526236,-2.273718,-0.430267,-5,-19
2022-06-28,-0.324334,-0.365928,-0.101784,-5,-20


# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [49]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,NaN
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,12,NaN
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,NaN
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,NaN


### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [50]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,NaN
2022-06-20,NaN,0.241150,-0.215080,1.465648,12,0.0
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [51]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [52]:
# or just
pd.isna(df13)

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [53]:
# the original is still there
df13

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,NaN
2022-06-20,NaN,0.241150,-0.215080,1.465648,12,0.0
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [54]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,NaN
2022-06-20,NaN,0.241150,-0.215080,1.465648,12,0.0
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [55]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [56]:
# all rows/cols with missing data stripped
df_no_na

,A,B,C,D,F,G
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [57]:
# original still intact
df132

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,NaN
2022-06-20,NaN,0.241150,-0.215080,1.465648,12,0.0
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [58]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

,A,B,C,D,F,G
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.0
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.0


In [59]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

,A,B,C,D,F,G
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11,3141.592654
2022-06-20,3141.592654,0.241150,-0.215080,1.465648,12,0.000000
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13,0.000000
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14,0.000000


# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...
Here we'll explore these in the context of the align function - 'aligning' the indexes of two dataframes.
Later we'll see these again in the context of _merging_ two data frames.

## Create two datasets with _mismatched_ indexes

In [60]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [61]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [62]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a, b, a_title = "", b_title = ""):
    a_styler = a.style.set_table_attributes("style='display:inline'").set_caption(a_title)
    b_styler = b.style.set_table_attributes("style='display:inline'").set_caption(b_title)
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [63]:
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.601640,17
2022-01-02 00:00:00,0.632191,16
2022-01-03 00:00:00,0.678718,24
2022-01-04 00:00:00,0.653614,21
2022-01-05 00:00:00,0.212363,18
2022-01-06 00:00:00,0.681628,16
2022-01-07 00:00:00,0.374608,8
2022-01-08 00:00:00,0.425099,23
2022-01-09 00:00:00,0.080273,8
2022-01-10 00:00:00,0.831372,17


Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [64]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [65]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,1.114387,32.000000
2022-01-04 00:00:00,1.502735,43.000000
2022-01-05 00:00:00,0.274097,41.000000
2022-01-06 00:00:00,0.879459,19.000000
2022-01-07 00:00:00,1.233596,27.000000
2022-01-08 00:00:00,0.517127,31.000000
2022-01-09 00:00:00,0.728302,27.000000
2022-01-10 00:00:00,1.512794,35.000000


In [66]:
# sub
d4.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.243049,16.000000
2022-01-04 00:00:00,-0.195507,-1.000000
2022-01-05 00:00:00,0.150629,-5.000000
2022-01-06 00:00:00,0.483798,13.000000
2022-01-07 00:00:00,-0.484380,-11.000000
2022-01-08 00:00:00,0.333070,15.000000
2022-01-09 00:00:00,-0.567756,-11.000000
2022-01-10 00:00:00,0.149950,-1.000000


In [67]:
# mul
d5.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.295696,192.000000
2022-01-04 00:00:00,0.554997,462.000000
2022-01-05 00:00:00,0.013110,414.000000
2022-01-06 00:00:00,0.134847,48.000000
2022-01-07 00:00:00,0.321784,152.000000
2022-01-08 00:00:00,0.039121,184.000000
2022-01-09 00:00:00,0.052019,152.000000
2022-01-10 00:00:00,0.566515,306.000000


In [68]:
# div
d6.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,1.557876,3.000000
2022-01-04 00:00:00,0.769753,0.954545
2022-01-05 00:00:00,3.439983,0.782609
2022-01-06 00:00:00,3.445516,5.333333
2022-01-07 00:00:00,0.436104,0.421053
2022-01-08 00:00:00,4.619218,2.875000
2022-01-09 00:00:00,0.123873,0.421053
2022-01-10 00:00:00,1.220055,0.944444


### removing the NaN values from the resultant dataframe

In [69]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

,A,B
2022-01-03 00:00:00,1.114387,32.000000
2022-01-04 00:00:00,1.502735,43.000000
2022-01-05 00:00:00,0.274097,41.000000
2022-01-06 00:00:00,0.879459,19.000000
2022-01-07 00:00:00,1.233596,27.000000
2022-01-08 00:00:00,0.517127,31.000000
2022-01-09 00:00:00,0.728302,27.000000
2022-01-10 00:00:00,1.512794,35.000000


## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [70]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.601640,17.000000
2022-01-02 00:00:00,0.632191,16.000000
2022-01-03 00:00:00,0.678718,24.000000
2022-01-04 00:00:00,0.653614,21.000000
2022-01-05 00:00:00,0.212363,18.000000
2022-01-06 00:00:00,0.681628,16.000000
2022-01-07 00:00:00,0.374608,8.000000
2022-01-08 00:00:00,0.425099,23.000000
2022-01-09 00:00:00,0.080273,8.000000
2022-01-10 00:00:00,0.831372,17.000000


In [71]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.601640,17
2022-01-02 00:00:00,0.632191,16
2022-01-03 00:00:00,0.678718,24
2022-01-04 00:00:00,0.653614,21
2022-01-05 00:00:00,0.212363,18
2022-01-06 00:00:00,0.681628,16
2022-01-07 00:00:00,0.374608,8
2022-01-08 00:00:00,0.425099,23
2022-01-09 00:00:00,0.080273,8
2022-01-10 00:00:00,0.831372,17


In [72]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.601640,17
2022-01-02 00:00:00,0.632191,16
2022-01-03 00:00:00,0.678718,24
2022-01-04 00:00:00,0.653614,21
2022-01-05 00:00:00,0.212363,18
2022-01-06 00:00:00,0.681628,16
2022-01-07 00:00:00,0.374608,8
2022-01-08 00:00:00,0.425099,23
2022-01-09 00:00:00,0.080273,8
2022-01-10 00:00:00,0.831372,17


In [73]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.601640,17
2022-01-02 00:00:00,0.632191,16
2022-01-03 00:00:00,0.678718,24
2022-01-04 00:00:00,0.653614,21
2022-01-05 00:00:00,0.212363,18
2022-01-06 00:00:00,0.681628,16
2022-01-07 00:00:00,0.374608,8
2022-01-08 00:00:00,0.425099,23
2022-01-09 00:00:00,0.080273,8
2022-01-10 00:00:00,0.831372,17


In [74]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

,A,B
2022-01-03 00:00:00,0.678718,24.000000
2022-01-04 00:00:00,0.653614,21.000000
2022-01-05 00:00:00,0.212363,18.000000
2022-01-06 00:00:00,0.681628,16.000000
2022-01-07 00:00:00,0.374608,8.000000
2022-01-08 00:00:00,0.425099,23.000000
2022-01-09 00:00:00,0.080273,8.000000
2022-01-10 00:00:00,0.831372,17.000000
2022-01-11 00:00:00,nan,nan
2022-01-12 00:00:00,nan,nan


There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general *exclude* missing data

In [75]:
# arithmetic mean, for each column (axis = 0)
df.mean()

A     0.454331
B    -0.227045
C     0.224541
D     0.534724
F    15.500000
dtype: float64

In [76]:
# mean across a row (axis = 1)
df.mean(1)

2022-06-19    1.975655
2022-06-20    2.626949
2022-06-21    2.654145
2022-06-22    3.051904
2022-06-23    3.103118
2022-06-24    2.953104
2022-06-25    3.831547
2022-06-26    4.186637
2022-06-27    4.536091
2022-06-28    4.053953
Freq: D, dtype: float64

## Apply

In [77]:
# apply custom lambdas
df.apply(lambda x: x.max()-x.min())

A    4.194737
B    4.528719
C    2.884895
D    2.223630
F    9.000000
dtype: float64

In [78]:
df

,A,B,C,D,F
2022-06-19,-0.494255,-0.988936,0.599886,-0.238421,11
2022-06-20,-0.356973,0.241150,-0.215080,1.465648,12
2022-06-21,1.819942,-2.255001,-0.269409,0.975193,13
2022-06-22,2.039595,0.388154,-0.583221,-0.585011,14
2022-06-23,-0.122144,-1.784505,1.192836,1.229404,15
2022-06-24,1.084618,-1.302547,-0.787501,-0.229048,16
2022-06-25,-2.155141,0.576865,2.097393,1.638619,17
2022-06-26,0.877102,0.214728,0.742553,1.098801,18
2022-06-27,1.526236,2.273718,-0.430267,0.310771,19
2022-06-28,0.324334,0.365928,-0.101784,-0.318715,20


In [79]:
# each subsequent value is a sum of all values before it in the respective column
cumsum_df = df.apply(np.cumsum)
render_df_side_by_side(df, cumsum_df, "OG Dataframe", "Cumulative Sum")

,A,B,C,D,F
2022-06-19 00:00:00,-0.494255,-0.988936,0.599886,-0.238421,11
2022-06-20 00:00:00,-0.356973,0.241150,-0.215080,1.465648,12
2022-06-21 00:00:00,1.819942,-2.255001,-0.269409,0.975193,13
2022-06-22 00:00:00,2.039595,0.388154,-0.583221,-0.585011,14
2022-06-23 00:00:00,-0.122144,-1.784505,1.192836,1.229404,15
2022-06-24 00:00:00,1.084618,-1.302547,-0.787501,-0.229048,16
2022-06-25 00:00:00,-2.155141,0.576865,2.097393,1.638619,17
2022-06-26 00:00:00,0.877102,0.214728,0.742553,1.098801,18
2022-06-27 00:00:00,1.526236,2.273718,-0.430267,0.310771,19
2022-06-28 00:00:00,0.324334,0.365928,-0.101784,-0.318715,20


## Histogramming

Frequencies, 'nuff said

```value_counts()```

In [80]:
series1 = pd.Series(np.random.randint(0,5, size = 25))
series1.value_counts()

4    7
1    6
0    4
2    4
3    4
dtype: int64

## String Methods

In [81]:
# build a series
some_string = "SERIes and Index are EqUIppeD WITh A seT Of stRINg PROCESSING METHoDs In tHe ```sTR``` attrIBute THAT MakE IT eASy To oPerATE ON EACh ELEmeNt OF The aRrAY. NOtE thAt PAtTERN-maTching in ```sTR``` gENeralLY UsES reGUlAr eXpreSsiONs bY DEfault (aND In Some cases AlwayS uSEs tHeM)."
str_series = pd.Series(some_string.split()) # split splits on whitespace by default

In [82]:
# manipulating the case - lower, upper etc.
# also the length of each string, split, replace, yada yada...
# very important when cleaning data (column names may have stupid whitespace, bad case, spacing etc.)
low = str_series.str.lower()
up = str_series.str.upper()

low_up_df = pd.DataFrame(
    {
        'og': str_series,
        'length': str_series.str.len(),
        'low': low,
        'up': up,
        'split_low_on_i': low.str.split('i', expand = False), # expand = True will break this usecase, try elsewhere
        'replace_a_with_star_in_low': low.str.replace('a','*', case=False, regex=True) # yeah, you can mess with regular expns
    })

low_up_df.tail(10)

,og,length,low,up,split_low_on_i,replace_a_with_star_in_low
36,eXpreSsiONs,11,expressions,EXPRESSIONS,"[express, ons]",expressions
37,bY,2,by,BY,[by],by
38,DEfault,7,default,DEFAULT,[default],def*ult
39,(aND,4,(and,(AND,[(and],(*nd
40,In,2,in,IN,"[, n]",in
41,Some,4,some,SOME,[some],some
42,cases,5,cases,CASES,[cases],c*ses
43,AlwayS,6,always,ALWAYS,[always],*lw*ys
44,uSEs,4,uses,USES,[uses],uses
45,tHeM).,6,them).,THEM).,[them).],them).


See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-string-methods) for more string operations.

# Merge

Combine series and dataframes, use set logic for indexes and relational algebra for joins/merges

## Concat

Chill, we've done this before. The same join = inner/outer/left/right drill. 

In [83]:
# let's make some more dataframes
concatDF = pd.DataFrame(np.random.randn(10,4)) # 10 rows and 4 columns of random numbers
pieces = [concatDF[:3], concatDF[3:7], concatDF[7:]]
reconstructedDF = pd.concat(pieces)
display('broken DFs: ',pieces)
render_df_side_by_side(concatDF, reconstructedDF, "OG", "Concatenated")

'broken DFs: '

[          0         1         2         3
 0  1.315844 -0.130505 -1.681361  1.449769
 1 -1.193995  0.853159 -1.039541 -0.186835
 2 -0.160908 -0.134667  0.890574  2.395770,
           0         1         2         3
 3  0.773639 -0.234214  1.229374 -0.464821
 4 -2.223670 -0.923040 -1.131311  0.401642
 5 -0.951100 -1.148039  0.506326  1.572006
 6  0.565847 -1.113986  0.411488 -0.123707,
           0         1         2         3
 7 -0.744027  1.422074 -0.791689 -1.557400
 8  0.331874  0.735312 -0.079832 -0.074715
 9 -0.670001 -0.681433 -2.069118  0.052029]

,0,1,2,3
0,1.315844,-0.130505,-1.681361,1.449769
1,-1.193995,0.853159,-1.039541,-0.186835
2,-0.160908,-0.134667,0.890574,2.395770
3,0.773639,-0.234214,1.229374,-0.464821
4,-2.223670,-0.923040,-1.131311,0.401642
5,-0.951100,-1.148039,0.506326,1.572006
6,0.565847,-1.113986,0.411488,-0.123707
7,-0.744027,1.422074,-0.791689,-1.557400
8,0.331874,0.735312,-0.079832,-0.074715
9,-0.670001,-0.681433,-2.069118,0.052029


_From the [page](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#concat)_:
Adding a column to a DataFrame is relatively fast. However, adding a row requires a copy, and may be expensive. We recommend passing a pre-built list of records to the DataFrame constructor instead of building a DataFrame by iteratively appending records to it.

## Join
SQL style merges. 

In [84]:
# two data frames
sl = pd.Series(np.random.randint(10, size=6))
sr = sl.shift(2).fillna(0).astype(int)
left = pd.DataFrame(
    {
        'key': list('ABCDEF'),
        'lval': sl
    }
)

right = pd.DataFrame(
    {
        'key': list('ABBCDE'),
        'rval': sr
    }
)

In [85]:
render_df_side_by_side(left, right, 'left', 'right')

,key,lval
0,A,0
1,B,9
2,C,9
3,D,1
4,E,8
5,F,6
,key,rval
0,A,0
1,B,0
2,B,0


In [86]:
result = pd.merge(left, right, on='key')
result

,key,lval,rval
0,A,0,0
1,B,9,0
2,B,9,0
3,C,9,9
4,D,1,9
5,E,8,1


# Grouping

"Group By" really means one or more of the following happening:
1. **Split**: data is broken into groups based on some criteria
2. **Apply**: a data operation is executed on each of those groups
3. **Combine**: the results are combined back into a data structure

In [87]:
grp_df = pd.DataFrame(
    {
        'colA': list('AABCAABDAA'),
        'colB': list('1234567890'),
        'colC': np.random.randn(10),
        'colD': np.random.randn(10)
    }
)
grp_df

,colA,colB,colC,colD
0,A,1,0.969012,0.728565
1,A,2,0.068391,0.490976
2,B,3,0.440808,0.546845
3,C,4,-0.023840,0.214648
4,A,5,-0.005095,1.804839
5,A,6,-1.258433,0.928127
6,B,7,-2.070305,1.042526
7,D,8,1.127938,1.620825
8,A,9,-0.456405,-0.238551
9,A,0,-0.333067,1.211010


In [88]:
# a DataFrameGroupBy object object
grpby = grp_df.groupby('colA')

# list all the groups found
g = grpby.groups
g

{'A': [0, 1, 4, 5, 8, 9], 'B': [2, 6], 'C': [3], 'D': [7]}

In [89]:
# to create a dataframe from the group use something like
grpby.get_group('A')

,colA,colB,colC,colD
0,A,1,0.969012,0.728565
1,A,2,0.068391,0.490976
4,A,5,-0.005095,1.804839
5,A,6,-1.258433,0.928127
8,A,9,-0.456405,-0.238551
9,A,0,-0.333067,1.211010


### Find all the groups

In [90]:
# but we may not know all the groups in the dataframe.
# so got to find a way to list all groups as dataframes.
# 
# some basic python helps create a list of groupby names
grpby_labels_list = [*g.keys()] # using the unpacking operator *
# alternatively
# grpby_labels_list = list(g.keys())
grpby_labels_list

['A', 'B', 'C', 'D']

In [91]:
# create a dict where the group label is the key and the group dataframe is the value
grps = {lbl: grpby.get_group(lbl) for lbl in grpby_labels_list}

# now use the dict to access individual dataframes created out of the groupby operation
grps['B']

# question: can we create a dataframe of dataframes?

,colA,colB,colC,colD
2,B,3,0.440808,0.546845
6,B,7,-2.070305,1.042526


In [92]:
# You can see the datatype of the value is a pandas dataframe
type(grps['A'])

pandas.core.frame.DataFrame

In [93]:
# make a function of it all...
def get_all_groups(df_grpby):
    return {lbl: df_grpby.get_group(lbl) for lbl in [*df_grpby.groups.keys()]}

In [94]:
grps = get_all_groups(grpby)
grps

{'A':   colA colB      colC      colD
 0    A    1  0.969012  0.728565
 1    A    2  0.068391  0.490976
 4    A    5 -0.005095  1.804839
 5    A    6 -1.258433  0.928127
 8    A    9 -0.456405 -0.238551
 9    A    0 -0.333067  1.211010,
 'B':   colA colB      colC      colD
 2    B    3  0.440808  0.546845
 6    B    7 -2.070305  1.042526,
 'C':   colA colB     colC      colD
 3    C    4 -0.02384  0.214648,
 'D':   colA colB      colC      colD
 7    D    8  1.127938  1.620825}

In [95]:
# get a sum of values from each group
grp_df.groupby('colA').sum()

,colC,colD
colA,,
A,-1.015597,4.924966
B,-1.629497,1.589371
C,-0.023840,0.214648
D,1.127938,1.620825


In [96]:
# let's add one more column to the dataframe
# we can try grouping on two columns
grp_df['colE'] = list('1231231231')
grp_df

,colA,colB,colC,colD,colE
0,A,1,0.969012,0.728565,1
1,A,2,0.068391,0.490976,2
2,B,3,0.440808,0.546845,3
3,C,4,-0.023840,0.214648,1
4,A,5,-0.005095,1.804839,2
5,A,6,-1.258433,0.928127,3
6,B,7,-2.070305,1.042526,1
7,D,8,1.127938,1.620825,2
8,A,9,-0.456405,-0.238551,3
9,A,0,-0.333067,1.211010,1


In [97]:
g2 = grp_df.groupby(['colA','colE'])
grps2 = get_all_groups(g2)
grps2

{('A',
  '1'):   colA colB      colC      colD colE
 0    A    1  0.969012  0.728565    1
 9    A    0 -0.333067  1.211010    1,
 ('A',
  '2'):   colA colB      colC      colD colE
 1    A    2  0.068391  0.490976    2
 4    A    5 -0.005095  1.804839    2,
 ('A',
  '3'):   colA colB      colC      colD colE
 5    A    6 -1.258433  0.928127    3
 8    A    9 -0.456405 -0.238551    3,
 ('B',
  '1'):   colA colB      colC      colD colE
 6    B    7 -2.070305  1.042526    1,
 ('B',
  '3'):   colA colB      colC      colD colE
 2    B    3  0.440808  0.546845    3,
 ('C',
  '1'):   colA colB     colC      colD colE
 3    C    4 -0.02384  0.214648    1,
 ('D',
  '2'):   colA colB      colC      colD colE
 7    D    8  1.127938  1.620825    2}

In [98]:
render_df_side_by_side(grps2[('A','1')], grps2[('A','2')], 'A,1', 'A,2')

,colA,colB,colC,colD,colE
0,A,1,0.969012,0.728565,1
9,A,0,-0.333067,1.211010,1
,colA,colB,colC,colD,colE
1,A,2,0.068391,0.490976,2
4,A,5,-0.005095,1.804839,2
